In [1]:
from digipat.data.augmentations.builder import N, Probability, AugmentationBuilder
from digipat.data.augmentations import *

import torch

aug = (
    AugmentationBuilder()
    + (N() + DarkSpotsAugmentation()) * 5
    + (Probability() + DarkSpotsAugmentation() + FatAugmentation()) * 2
    + SquamousAugmentation()
).build()

aug(torch.zeros((300,300,3)), torch.zeros((300,300)))

AttributeError: 'numpy.ndarray' object has no attribute 'permute'